# TP 2 de Statistique descriptive avec SAS

### Les formats

Un format est un *masque d'affichage des données*. Cet objet permet en SAS de faciliter la lecture des tables. 

Il existe deux types de format : les formats (aussi appelés les formats de lecture) et les informats (ou format d'écriture).

L'exemple typique d'utilisation de format est celui des formats de date. Par défaut, SAS stocke les dates sous la forme d'un entier qui correspond au nombre de jours entre la date et le 1er janvier 1960. Il s'agit donc d'une variable numérique. Il est très délicat pour un utilisateur de manipuler ces variables sous cette forme (à quoi correspond par exemple la date -4017?). Les formats de date permettent de changer l'affichage de la variable : on peut par exemple choisir d'afficher la date sous la forme Jour/Mois/Année à l'aide du format DDMMYY. ou sous la forme MoisAnnée à l'aide du format MONYY.

Les formats ne s'appliquent pas uniquement sur les dates mais sur n'importe quelle variable. On peut par exemple vouloir afficher des numéros de départements sur trois positions en complétant par des zéros à gauche (001, 002 ...) ou afficher un nombre restreint de décimales.

Une liste des formats est disponible ici.

Pour appliquer un format à une variable, on utilisera l'instruction FORMAT dans une procédure ou dans une étape DATA.

Par exemple, prenons la table AIR de la bibliothèque SASHELP.

Il est possible de créer 

## Exercice 1 : Analyse de l'âge et de la catégorie socioprofessionnelle des habitants d'Ille-et-Vilaine

**Question 1** : Créez une bibliothèque *source* en lecture seule et une bibliothèque *resultat* ayant vocation à contenir les résultats de vos analyses.

Je pense que certains étudiants ont encore des difficultés avec ça...

In [ ]:
LIBNAME source "/folders/myfolders/Donnees_TP_SAS/TP2/Source" ACCESS = READONLY;
LIBNAME resultat "/folders/myfolders/Donnees_TP_SAS/TP2/Resultat" ;

In [ ]:
PROC PRINT DATA = source.rp12_d35(obs = 20);
RUN;

**Question 2** : Créez un format NOMTRANCHE qui sera appliqué à la variable AGE tel qu'on associe :
* Si $\text{AGE} \in [00;18[" \longrightarrow$ "Enfants"
* Si $\text{AGE} \in [18;26[" \longrightarrow$ "Etudiants"
* Si $\text{AGE} \in [26;40[" \longrightarrow$ "Jeunes actifs"
* Si $\text{AGE} \in [40;55[" \longrightarrow$ "Actifs"
* Si $\text{AGE} \in [55;65[" \longrightarrow$ "Actifs expérimentés"
* Si $\text{AGE} \in [65;80[" \longrightarrow$ "Jeunes seniors"
* Si $\text{AGE} \in [80;+\infty[" \longrightarrow$ "Seniors"

Où est stocké le format NOMTRANCHE ?

In [ ]:
PROC FORMAT;
    VALUE NOMTRANCHE
        LOW - 17 = "Enfants"
        18 - 25 = "Etudiants"
        26 - 39 = "Jeunes actifs"
        40 - 54 = "Actifs"
        55 - 64 = "Actifs expérimentés"
        65 - 79 = "Jeunes seniors"
        80 - HIGH = "Seniors";
RUN;

**Question 3** : Appliquez le format NOMTRANCHE à la variable AGETRANCHE.

In [ ]:
OPTIONS FMTSEARCH=(work);

DATA resultat.rp12_d35;
SET resultat.rp12_d35;
FORMAT AGE NOMTRANCHE.;
RUN;

**Question 4** : Vérifiez le type de la variable AGE à l'aide d'une procédure.

In [ ]:
PROC CONTENTS DATA = resultat.rp12_d35;
RUN;

**Question 4** : Importez les formats NOMTACT et NOMCS.

In [ ]:
OPTIONS FMTSEARCH=(work source);

**Question 5** : À l'aide de la procédure FORMAT, récupérez des informations sur les formats NOMTACT et NOMCS.

In [ ]:
PROC FORMAT LIBRARY = source FMTLIB;
RUN;

**Question 6** : Affichez un tableau de contingence des variables AGETRANCHE et TACT en utilisant l'instruction FORMAT de la procédure FREQ. On appliquera les formats NOMTRANCHE et NOMTACT.

In [ ]:
PROC FREQ DATA = resultat.RP12_d35;
TABLE AGE * TACT / NOPCT NOROWPCT NOCOLPCT;
FORMAT AGE NOMTRANCHE.
    TACT $NOMTACT.;
WEIGHT IPONDI;
RUN;

**Question 7** : Affichez le tableau des fréquences conditionnelles par ligne (aussi appelé tableau des profils-lignes) du tableau de contingence précédent.

Quelles sont les types d'activités sur-représentés chez les étudiants ? Quid chez les seniors ?

In [ ]:
PROC FREQ DATA = resultat.RP12_d35;
TABLE AGE * TACT / NOCOL NOFREQ;
FORMAT AGE NOMTRANCHE.
    TACT $NOMTACT.;
WEIGHT IPONDI;
RUN;

**Question 8** : Rappelez la définition de la statistique du Khi-Deux. Dans le cas du tableau de contingence de la question 6, dans quel intervalle est comprise la statistique du Khi-Deux ?

Discutez du cas où la statistique est nulle. 

À l'aide d'une procédure FREQ, calculez la valeur de cette statistique. Pouvez vous conclure ? Utilisez un indicateur dérivé à la place.

**Réponse :**

La statistique du Khi-Deux est donnée ici par $\displaystyle \chi^2 = \sum_{i = 1}^7 \sum_{j = 1}^7 \frac{(n_{ij} - \frac{n_{i\bullet}n_{\bullet j}}{n})^2}{\frac{n_{i\bullet}n_{\bullet j}}{n}}$ où $n_{ij}$ désigne l'entrée (i,j) du tableau de contingence.

$\chi^2 \in [0; 6 = min(I-1, J-1)]$ (I et J désignant le nombre de modalités de chaque variable).

$\displaystyle \chi^2 = 0 \Longrightarrow n_{ij} = \frac{n_{i\bullet}n_{\bullet j}}{n}$ (car il s'agit d'une somme de termes positifs nulle) $\Longrightarrow f_{ij} = f_{i\bullet} f_{\bullet j}$. Il s'agit d'une situation où les deux variables qualitatives sont indépendantes.

Les étudiants n'ont pas encore vu que sous des hypothèses asymptotiques, la statistique du $\chi^2$ était distribué selon une loi qui porte le même nom et donc qu'ils peuvent réaliser un test d'hypothèse. À la place, ils ont des indicateurs comme le coefficient de contingence ou le V de Cramer.

In [ ]:
PROC FREQ DATA = resultat.RP12_d35;
TABLE AGE * TACT / NOCOL NOFREQ CHISQ;
WEIGHT IPONDI;
RUN;

## Exercice 2 : Application de formats externes à la base Pisa

## Exercice 3 : Analyse de données issues de l'enquête Formation, qualification et profession.